## Importações

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import shap

# Pré-processamento e pipelines
from transformers_credits import DateFeatureExtractor, CreditsCounter, MultiLabelProcessor, RareCategoryGrouper 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, StandardScaler

# Modelos de Regressão
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

# Validação e busca de hiperparâmetros
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error

c:\temp\ws-vscode\DSI-PISI3\projeto-pisi3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carregando dataset 

In [3]:
df = pd.read_csv('../data/filmes_filtrados_credits.csv')

## Dividindo os dados

In [4]:
TARGET = 'vote_average'
NUMERICAL_COLS = ['popularity', 'budget', 'runtime', 'credits_count'] 
CATEGORICAL_COL = ['original_language']
DATE_COLUMN = 'release_date'
MULTILABEL_COLS = ['genres', 'production_companies']
CREDITS_COL = 'credits'

features = ['popularity', 'budget', 'runtime', 'original_language', 'release_date', 'genres', 'production_companies', CREDITS_COL]
X = df[features]
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

## Construção do Pipeline

In [5]:
# Configuração para o processador de múltiplos rótulos
multilabel_config = {
    'genres': {'sep': '-', 'top_n': 12, 'prefix': 'genre'},
    'production_companies': {'sep': '-', 'top_n': 8, 'prefix': 'prod'}
}

# Pipeline para colunas numéricas (sem tratamento de outliers aqui, será feito no pipeline principal)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline para a coluna categórica
categorical_transformer = Pipeline(steps=[
    ('rare', RareCategoryGrouper(column='original_language', top_n=10)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pré-processador principal com ColumnTransformer
# Nota: 'remainder' passará as colunas não especificadas (que serão as processadas pelas etapas anteriores do pipeline)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERICAL_COLS),
        ('cat', categorical_transformer, CATEGORICAL_COL)
    ],
    remainder='passthrough' # Mantém as colunas processadas anteriormente (ano, mês, multilabel)
)

# Pipeline final
full_pipeline = Pipeline(steps=[
    ('credits_transform', CreditsCounter(credits_column=CREDITS_COL)),
    ('date_transform', DateFeatureExtractor(date_column=DATE_COLUMN)),
    ('multilabel_transform', MultiLabelProcessor(multilabel_cols_config=multilabel_config)),
    ('preprocess', preprocessor)
])

## Definição dos Modelos e Espaço de Busca

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=87)

models_to_tune = {
    "XGBoost": {
        "model": XGBRegressor(objective='reg:squarederror', random_state=87, n_jobs=-1),
        "params": {
            'regressor__n_estimators': randint(100, 300),
            'regressor__max_depth': randint(3, 10),
            'regressor__learning_rate': uniform(0.01, 0.2),
            'regressor__subsample': uniform(0.7, 0.3),
            'regressor__colsample_bytree': uniform(0.7, 0.3),
        }
    },
    "RandomForest": {
        "model": RandomForestRegressor(random_state=87, n_jobs=-1),
        "params": {
            'regressor__n_estimators': randint(100, 500),
            'regressor__max_depth': randint(5, 20),
            'regressor__min_samples_leaf': randint(1, 10),
        }
    },
    "SVR": {
        "model": SVR(),
        "params": {
            'regressor__C': loguniform(1e-1, 1e2),
            'regressor__epsilon': uniform(0.01, 0.3),
            'regressor__gamma': ['scale', 'auto']
        }
    }
}

## Treinamento, Otimização e Avaliação


In [7]:
results = {}
best_estimators = {}

for name, config in models_to_tune.items():
    print(f"--- Treinando e otimizando {name} ---")
    
    # Cria um pipeline específico para cada modelo
    model_pipeline = Pipeline(steps=[
        ('features', full_pipeline),
        ('regressor', config['model'])
    ])   
    rand_search = RandomizedSearchCV(
        estimator=model_pipeline,
        param_distributions=config['params'],
        n_iter=25, 
        cv=kf,
        scoring='r2',
        random_state=87,
        verbose=1,
        n_jobs=-1
    )
    
    rand_search.fit(X_train, y_train)
    
    best_estimator = rand_search.best_estimator_
    best_estimators[name] = best_estimator
    
    # Avaliação com os dados de teste
    y_pred = best_estimator.predict(X_test)
    
    metrics = {
        'Melhor R² (CV)': rand_search.best_score_,
        'R² (Teste)': r2_score(y_test, y_pred),
        'MAE (Teste)': mean_absolute_error(y_test, y_pred),
        'RMSE (Teste)': np.sqrt(mean_squared_error(y_test, y_pred)),
    }
    
    results[name] = metrics
    print(f"Melhores parâmetros para {name}: {rand_search.best_params_}\n")

# Exibir resultados
results_df = pd.DataFrame(results).T
print("\n--- Resultados Finais da Avaliação ---")
print(results_df)

--- Treinando e otimizando XGBoost ---
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Melhores parâmetros para XGBoost: {'regressor__colsample_bytree': np.float64(0.8696231377727208), 'regressor__learning_rate': np.float64(0.04328037357016285), 'regressor__max_depth': 5, 'regressor__n_estimators': 241, 'regressor__subsample': np.float64(0.8173221947525602)}

--- Treinando e otimizando RandomForest ---
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Melhores parâmetros para RandomForest: {'regressor__max_depth': 19, 'regressor__min_samples_leaf': 3, 'regressor__n_estimators': 354}

--- Treinando e otimizando SVR ---
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Melhores parâmetros para SVR: {'regressor__C': np.float64(3.9457066086519057), 'regressor__epsilon': np.float64(0.22353628374033715), 'regressor__gamma': 'auto'}


--- Resultados Finais da Avaliação ---
              Melhor R² (CV)  R² (Teste)  MAE (Teste)  RMSE (Teste)
XGBoost        

## Análise de Features com SHAP

In [1]:
# Supondo que o XGBoost seja o melhor modelo, vamos analisá-lo com SHAP
best_model_name = results_df['R² (Teste)'].idxmax()
final_model_pipeline = best_estimators[best_model_name]

print(f"\n--- Analisando o melhor modelo ({best_model_name}) com SHAP ---")

# Passo 1: Extrair o pipeline de features que já foi treinado
features_pipeline = final_model_pipeline.named_steps['features']

# Passo 2: Obter os nomes das features do pipeline já treinado
feature_names = features_pipeline.get_feature_names_out(X_train.columns)

# Passo 3: Transformar os dados de teste e criar um DataFrame com os nomes corretos
# (Boa prática para garantir que o SHAP associe os valores corretamente)
X_test_transformed = features_pipeline.transform(X_test)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)

# Passo 4: Criar o explainer SHAP
# Passamos apenas o modelo; o SHAP infere os nomes a partir do DataFrame
explainer = shap.Explainer(final_model_pipeline.named_steps['regressor'])

# Passo 5: Calcular os valores SHAP usando o DataFrame transformado
shap_values = explainer(X_test_transformed_df)

# Passo 6: Plotar os gráficos de resumo
print("\nGráfico de Importância das Features (média do impacto):")
plt.title(f"Importância das Features (SHAP) para o modelo {best_model_name}")
# A versão "bar plot" não precisa do argumento feature_names
shap.summary_plot(shap_values, plot_type="bar", max_display=15)
plt.show()

print("\nGráfico de Resumo (impacto e direção de cada feature):")
# Este gráfico usa o DataFrame para exibir os valores corretamente na legenda de cores
shap.summary_plot(shap_values, X_test_transformed_df, max_display=15)
plt.show()

NameError: name 'results_df' is not defined